In [ ]:
context = '[DOC] [TLE] J! Archive - Show #5243, aired 2007-05-30 [PAR] May 30, 2007 ... 1976: "A Single Colorado Mountain". $1200, 19. Founded in the 1530s, this \ncapital of Jalisco state is the second-largest in Mexico. $1200, 23... [DOC] [TLE] November 14, 2008 [PAR] ... "Regular Folks" Ordinary People 1932: "Magnificent Inn" Grand Hotel 1976: "A \nSingle Colorado Mountain" Rocky Down Mexico Way: In 1986 Mexico scored... [DOC] [TLE] LAND USE AND LANDSCAPE CHANGE IN THE COLORADO ... [PAR] ... landscape change in a single Colorado mountain valley using methods from \n..... and for multi-family 1970 1973 1976 1979 1982 1985 1988 1991 complexes... [DOC] [TLE] J! set #1 Flashcards | Quizlet [PAR] 18 shows included Learn with flashcards, games, and more  for free. '

from errudite.processor import SpacyAnnotator
spacy_annotator = SpacyAnnotator(disable=['parser', 'ner', 'textcat'])
list(spacy_annotator.process_text(context))

In [ ]:
import glob, os

import errudite
from errudite.io import DatasetReader
from errudite.predictors import Predictor
from errudite.targets.instance import Instance
from errudite.targets.label import Label
from errudite.utils import accuracy_score, normalize_file_path

DATASET_FOLDER = normalize_file_path("~/datasets/raw_data/mrqa/out_of_domain_devs")
sample_size=100000
file_path = ",".join(glob.glob(os.path.join(DATASET_FOLDER, "*.jsonl.gz")))
reader = DatasetReader.by_name("mrqa")(
    cache_folder_path=f"~/datasets/caches/dataset_debug/mrqa-{sample_size}")
instances = reader.read(file_path, sample_size=sample_size)

In [1]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import os
def import_sys():
    import sys
    sys.path.append('..')
    sys.path.append('../..')
import_sys()

import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)  # pylint: disable=invalid-name

In [2]:
import glob, os

import errudite
from errudite.io import DatasetReader
from errudite.predictors import Predictor
from errudite.targets.instance import Instance
from errudite.targets.label import Label
from errudite.utils import accuracy_score, normalize_file_path

DATASET_FOLDER = normalize_file_path("~/datasets/raw_data/mrqa/out_of_domain_devs")
sample_size=10
file_path = ",".join(glob.glob(os.path.join(DATASET_FOLDER, "*.jsonl.gz")))
reader = DatasetReader.by_name("mrqa")(
    cache_folder_path=f"~/datasets/caches/dataset_debug/mrqa-{sample_size}")
instances = reader.read(file_path, sample_size=sample_size)

MODEL_FOLDER = normalize_file_path("~/datasets/models/mrqa/")
predictor = Predictor.by_name("mrqa")(
    name="mrqa_path", 
    model_path=os.path.join(MODEL_FOLDER, "mrqa_bert_base.gz"))

INFO:pytorch_pretrained_bert.modeling:Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
/Users/tongshuangwu/sourcetree/errudite_dataset_debug/venv/lib/python3.6/site-packages/sklearn/utils/linear_assignment_.py:21: DeprecationWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  DeprecationWarning)
INFO:errudite.utils.file_utils:Errudite cache folder selected: /Users/tongshuangwu/datasets/caches/dataset_debug/mrqa-10
INFO:errudite.io.dataset_reader:Reading instances from lines in file at: /Users/tongshuangwu/datasets/raw_data/mrqa/out_of_domain_devs/DuoRC.jsonl.gz
INFO:errudite.io.mrqa_reader:Reading from dataset: DuoRC.ParaphraseRC.
1it [00:00,  2.71it/s]
INFO:allennlp.models.archival:loading archive file /Users/tongshuangwu/datasets/models/mrqa/mrqa_bert_base.gz
INFO:allennlp.models.archival:extracting archive file /Users/tongshuangwu/datasets/models/mrqa

In [6]:
import pandas as pd
from tqdm import tqdm
predictors = { p.name: p for p in [predictor] }
predictions = { p: [] for p in predictors }
logger.info("Running predictions....")
for instance in tqdm(instances):
    instance_predictions = []
    for predictor in predictors.values():
        prediction = Predictor.by_name("qa_task_class").model_predict(
            predictor, 
            instance.question, 
            instance.context, 
            instance.groundtruths)
        instance_predictions.append(prediction)
        #print(prediction.doc, prediction.span_start, prediction.span_end, instance.context)
        predictions[predictor.name].append(prediction)
    instance.set_entries(predictions=instance_predictions)
for predictor in predictors.values():
    predictor.evaluate_performance(instances)
print(pd.DataFrame([ {"predictor": p.name, "f1": p.perform["f1"] } for p in predictors.values() ]))
Instance.build_instance_hashes(instances)

INFO:__main__:Running predictions....
  0%|          | 0/10 [00:00<?, ?it/s]

[array([{'original_passage': 'What was Jill\'s mother\'s face burned by? [SEP] \n\nNew Orleans, Louisiana, 1927. An enraged posse of men descend on the isolated Seven Doors Hotel deep in the swamps. They grab an artist called Schweik (Antoine Saint John), who is cloistered there. Accusing him of being a warlock, Schweik is dragged down to the cellar where he is savagely beaten with heavy chains, tortured with quicklime acid, and crucified with his wrists nailed to a cellar wall, despite his dire warnings of evil to be unleashed.New Orleans, 1981. Liza Merril (Catriona MacColl) is a young woman who arrives from New York City to claim the hotel as her inheritance. No sooner has architect friend Marin Avery (Michele Mirabella) begins to show her around the property, strange incidents begin to happen. A painter (Anthony Flees) falls off his rig and is horribly injured, coughing up blood and babbling about, "the eyes, the eyes." Dr. John McCabe (David Warbeck) arrives to take the injured ma

 10%|█         | 1/10 [00:09<01:27,  9.70s/it]

[array([{'original_passage': 'What was Jill\'s mother\'s face burned by? [SEP] \n\nNew Orleans, Louisiana, 1927. An enraged posse of men descend on the isolated Seven Doors Hotel deep in the swamps. They grab an artist called Schweik (Antoine Saint John), who is cloistered there. Accusing him of being a warlock, Schweik is dragged down to the cellar where he is savagely beaten with heavy chains, tortured with quicklime acid, and crucified with his wrists nailed to a cellar wall, despite his dire warnings of evil to be unleashed.New Orleans, 1981. Liza Merril (Catriona MacColl) is a young woman who arrives from New York City to claim the hotel as her inheritance. No sooner has architect friend Marin Avery (Michele Mirabella) begins to show her around the property, strange incidents begin to happen. A painter (Anthony Flees) falls off his rig and is horribly injured, coughing up blood and babbling about, "the eyes, the eyes." Dr. John McCabe (David Warbeck) arrives to take the injured ma

 20%|██        | 2/10 [00:19<01:18,  9.85s/it]

...

 90%|█████████ | 9/10 [01:32<00:10, 10.30s/it]

[array([{'original_passage': 'Who transports himself to Bermuda? [SEP] \n\nIn olden times, England is in turmoil. With the death of the King, noone can decide who is the rightful heir to the throne. With war threatening to tear the country asunder, a stone and anvil appear from the heavens in London town, with a sword planted firmly in the anvil. On the hilt of the sword, read the words, "Whoso pulleth out this sword of this stone and anvil is rightwise king, born of England." Many try to pull the sword, but none can budge it. In time, the sword is forgotten, and the Dark Ages plunge the countryside into a dismal state.Out in the woods one day, Sir Kaye is out hunting with his adopted brother Arthur, whom everyone names \'Wart.\' After causing Kaye to miss shooting a deer, Arthur promises to retrieve the arrow. Arthur\'s journey leads him to a small hut, wherein resides a sorcerer named Merlin, and his pet owl, Archimedes, who can talk. Merlin explains to Arthur that he was expecting h

100%|██████████| 10/10 [01:44<00:00, 10.64s/it]

[array([{'original_passage': 'Where did the sword appear to be [SEP] \n\nIn olden times, England is in turmoil. With the death of the King, noone can decide who is the rightful heir to the throne. With war threatening to tear the country asunder, a stone and anvil appear from the heavens in London town, with a sword planted firmly in the anvil. On the hilt of the sword, read the words, "Whoso pulleth out this sword of this stone and anvil is rightwise king, born of England." Many try to pull the sword, but none can budge it. In time, the sword is forgotten, and the Dark Ages plunge the countryside into a dismal state.Out in the woods one day, Sir Kaye is out hunting with his adopted brother Arthur, whom everyone names \'Wart.\' After causing Kaye to miss shooting a deer, Arthur promises to retrieve the arrow. Arthur\'s journey leads him to a small hut, wherein resides a sorcerer named Merlin, and his pet owl, Archimedes, who can talk. Merlin explains to Arthur that he was expecting him

AttributeError: 'list' object has no attribute 'show_instance'

In [7]:
for predictor in predictors.values():
    predictor.evaluate_performance(instances)
print(pd.DataFrame([ {"predictor": p.name, "f1": p.perform["f1"] } for p in predictors.values() ]))

         f1  predictor
0  0.557143  mrqa_path


[autoreload of errudite.predictors.qa.mrqa_allennlp.BERT_QA failed: Traceback (most recent call last):
  File "/Users/tongshuangwu/sourcetree/errudite_dataset_debug/venv/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/Users/tongshuangwu/sourcetree/errudite_dataset_debug/venv/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 434, in superreload
    module = reload(module)
  File "/Users/tongshuangwu/sourcetree/errudite_dataset_debug/venv/lib/python3.6/imp.py", line 315, in reload
    return importlib.reload(module)
  File "/Users/tongshuangwu/sourcetree/errudite_dataset_debug/venv/lib/python3.6/importlib/__init__.py", line 166, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 618, in _exec
  File "<frozen importlib._bootstrap_external>", line 678, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  

In [ ]:
import pandas as pd
pd.DataFrame([ {"predictor": p.name, "f1": p.perform["f1"] } for p in predictors.values() ])

In [ ]:
TRAIN_DATASET_FOLDER = normalize_file_path("~/datasets/raw_data/mrqa/trains")
sample_size=100
file_path = ",".join(glob.glob(os.path.join(DATASET_FOLDER, "*.jsonl.gz"))[:2])
reader.count_vocab_freq(file_path)